In [7]:
%use kandy
%use dataframe

Define basic properties for simulation.
'startDate' and 'endDate' could be changed because of all assets' availability.

In [14]:
import portfolio.rebalancer.strategy.HAAStrategy
import java.time.ZoneId
import java.time.ZonedDateTime

val strategy = HAAStrategy()
val initialBudget = 10000.0
val startDate = ZonedDateTime.of(1970, 1, 1, 0, 0, 0, 0, ZoneId.systemDefault())
val endDate = ZonedDateTime.of(2024, 1, 1, 0, 0, 0, 0, ZoneId.systemDefault())

Load all the CSV files including price data of various ETFs

In [12]:
import portfolio.rebalancer.io.PriceDataManager

val csvFilesDir = System.getProperty("user.dir") + "../csv"
val allPrices = PriceDataManager(baseDir = csvFilesDir).allPrices

/Users/sh.hwang/personal/quant/src/backtests/notebook/../csv


{2003-12-06T06:00+09:00[Asia/Seoul]={TIP=54.149002, EFA=24.54471, EEM=11.491741, IEF=49.647411, QQQ=29.922518, IWM=40.933609, TLT=44.950962, SPY=72.535614}, 2003-12-09T06:00+09:00[Asia/Seoul]={TIP=54.101166, EFA=24.750179, EEM=11.514285, IEF=49.460117, QQQ=30.093306, IWM=41.237946, TLT=44.636604, SPY=73.024406}, 2003-12-10T06:00+09:00[Asia/Seoul]={TIP=53.962948, EFA=24.646505, EEM=11.466196, IEF=49.214272, QQQ=29.401613, IWM=40.541767, TLT=44.322311, SPY=72.460922}, 2003-12-11T06:00+09:00[Asia/Seoul]={TIP=54.106468, EFA=24.469311, EEM=11.420367, IEF=49.337215, QQQ=29.512621, IWM=40.176548, TLT=44.395615, SPY=72.454124}, 2003-12-12T06:00+09:00[Asia/Seoul]={TIP=54.31913, EFA=24.740761, EEM=11.628485, IEF=49.6357, QQQ=30.144541, IWM=41.298805, TLT=44.715206, SPY=73.268776}, 2003-12-13T06:00+09:00[Asia/Seoul]={TIP=54.276619, EFA=24.82181, EEM=11.681829, IEF=49.6357, QQQ=30.093306, IWM=41.594662, TLT=44.799061, SPY=73.411339}, 2003-12-16T06:00+09:00[Asia/Seoul]={TIP=54.159668, EFA=24.655928

Simulate HAA strategy in as longest period where all the assets are available as possible

In [25]:
import portfolio.rebalancer.StrategySimulator
import portfolio.rebalancer.dto.SymbolPricesByDate

val strategySimulator = StrategySimulator()
val result = strategySimulator.simulate(
    budget = initialBudget,
    strategy = strategy,
    allTimeSymbolPrices = SymbolPricesByDate(value = allPrices),
    startDate = startDate,
    endDate = endDate,
)

Start simulating from 2007-05-31T06:00+09:00[Asia/Seoul] with 10000.0 USD
datesToReBalance=[2008-05-31T06:00+09:00[Asia/Seoul], 2008-07-01T06:00+09:00[Asia/Seoul], 2008-07-30T06:00+09:00[Asia/Seoul], 2008-08-30T06:00+09:00[Asia/Seoul], 2008-09-30T06:00+09:00[Asia/Seoul], 2008-10-30T06:00+09:00[Asia/Seoul], 2008-12-02T06:00+09:00[Asia/Seoul], 2008-12-30T06:00+09:00[Asia/Seoul], 2009-01-30T06:00+09:00[Asia/Seoul], 2009-02-28T06:00+09:00[Asia/Seoul], 2009-03-28T06:00+09:00[Asia/Seoul], 2009-04-28T06:00+09:00[Asia/Seoul], 2009-05-28T06:00+09:00[Asia/Seoul], 2009-06-30T06:00+09:00[Asia/Seoul], 2009-07-28T06:00+09:00[Asia/Seoul], 2009-08-28T06:00+09:00[Asia/Seoul], 2009-09-29T06:00+09:00[Asia/Seoul], 2009-10-28T06:00+09:00[Asia/Seoul], 2009-11-28T06:00+09:00[Asia/Seoul], 2009-12-29T06:00+09:00[Asia/Seoul], 2010-01-28T06:00+09:00[Asia/Seoul], 2010-03-02T06:00+09:00[Asia/Seoul], 2010-03-30T06:00+09:00[Asia/Seoul], 2010-04-28T06:00+09:00[Asia/Seoul], 2010-05-28T06:00+09:00[Asia/Seoul], 2010-06-

Total profit rate

In [27]:
"${(result.timeSeriesData.last().second / initialBudget - 1.0) * 100}% between ${result.timeSeriesData.first().first} and ${result.timeSeriesData.last().first}"

305.66015698000024% between 2007-05-31T06:00+09:00[Asia/Seoul] and 2023-12-30T06:00+09:00[Asia/Seoul]

Plot a line graph showing total asset over time

In [24]:
import java.time.format.DateTimeFormatter

val formatter = DateTimeFormatter.ofPattern("yyyy-MM-dd")

val dates = result.timeSeriesData.map { it.first.format(formatter) }
val assets = result.timeSeriesData.map { it.second }

mapOf(
    "date" to dates,
    "asset" to assets,
).plot {
    layout { 
        title = "Total asset over time"
        size = 1200 to 400
     }
    line {
        x(dates)
        y(assets)
        color = Color.PURPLE
        width = 1.0
    }
}

Drawdowns over time

In [50]:
val dates = result.timeSeriesData.map { it.first.format(formatter) }.drop(1)
val drawdowns = DrawdownCalculator().calculateDrawdowns(result.timeSeriesData.map { it.second }).map { it * 100 }
val mdds = drawdowns.scan(0.0) { acc, x -> minOf(acc, x) }.drop(1)

mapOf(
    "date" to dates + dates,
    "drawdown" to drawdowns + mdds,
    "line" to List(dates.size) { "Drawdown" } + List(dates.size) { "MDD" },
).plot {
    groupBy("line") {
        line {
            x("date")
            y("drawdown")
            color("line")
        }
    }
    layout {
        title = "Drawdown and MDD over time"
        size = 1200 to 400
    }
}

Max drawdown